# 2.1 Aggregation

Aggregate functions are used to calculate expressions over a set of values.

RDFox offers several, including common functions like SUM and COUNT, and more specialized functions like COUNT_MAX and MAX_ARGMIN.

See the [full list shown here](https://oxfordsemtech.github.io/DocumentationDevBuild/querying.html#aggregate-functions) (excluding non-deterministic functions).

# Using aggregate functions in rules

To use aggregation in a rule, you must declare it within the body with `AGGREGATE()`, that contains the relevant atoms.

You must also indicate which variable(s) the you're aggregating on with `ON`, and bind the result to a new variable with `BIND`.

# Variable scope

Variables in aggregate atoms are local to the atom unless mentioned outside of the atom. Two aggregates can use a variable with the same name so long as it is not a mentioned group variable.


## Example

Below is an example of MAX and COUNT_MAX used within one rule to calculate the percentage of 5 star reviews.

In [50]:
agg_data = """
@prefix : <https://rdfox.com/example#> .

:review1 a :Review ;
    :hasStars 5 .

:review2 a :Review ;
    :hasStars 5 .

:review3 a :Review ;
    :hasStars 2 .

:reviewLog :hasReview :review1 ;
    :hasReview :review2 ;
    :hasReview :review3 .
    
"""

In [52]:
agg_rules = """
[?reviewLog, :hasPercentageOfFiveStars, ?percentage] :-
    AGGREGATE (
        [?reviewLog, :hasReview, ?review]
        ON ?reviewLog
        BIND COUNT(?review) AS ?count 
    ),
    AGGREGATE (
        [?reviewLog, :hasReview, ?review],
        [?review, :hasStars, ?stars]
        ON ?reviewLog
        BIND COUNT_MAX(?stars) AS ?maxCount 
    ),
    BIND( (?maxCount / ?count) AS ?percentage ) .
"""

In [54]:
import requests

# Set up the SPARQL endpoint
rdfox_server = "http://localhost:12110"

# Helper function to raise exception if the REST endpoint returns an unexpected status code
def assert_response_ok(response, message):
    if not response.ok:
        raise Exception(
            message + "\nStatus received={}\n{}".format(response.status_code, response.text))

# Add data
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=agg_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get rules
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=agg_rules)
assert_response_ok(rules_response, "Failed to add rule.")

# Issue select query
sparql_text = "SELECT ?percentage WHERE { ?review :hasPercentageOfFiveStars ?percentage}"
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": sparql_text})
assert_response_ok(response, "Failed to run select query.")
print('\n=== Percentage of 5 star ratings ===')
print(response.text)



=== Percentage of 5 star ratings ===
?percentage
0.666666666666666667
1.0



## Exercise

...